In [1]:
import random
from tqdm import tqdm
from datasets import load_dataset, Dataset

In [2]:
ds = load_dataset("Elfsong/Venus_t", "python3", split="train")

In [3]:
prompt_template = """
Given the following problem details and the instruction, optimize the original solution.

# Instruction:
{instruction}

# Problem Description:
{content}

# Original Solution:
{solution_code}
"""

In [ ]:
dpo_dataset = list()

for instance in tqdm(list(ds)):
    if len(instance["rt_list"]) < 5:
        continue

    for _ in range(512):
        # Randomly sample three solutions
        (solution_1, solution_2, solution_3) = random.sample(instance["rt_list"], 3)

        # If the runtime difference is greater than 16 ms, generate a new training instance
        if (int(solution_1['runtime']) - int(solution_2['runtime']) >= 16) and (int(solution_2['runtime']) - int(solution_3['runtime']) >= 16):
            sft_instance = {
                "input": prompt_template.format(instruction="Generate a solution with faster runtime.", content=instance["content"], solution_code=solution_2["code"]),
                "chosen": solution_3["code"],
                "rejected": solution_1["code"]
            }
            dpo_dataset.append(sft_instance)

new_ds = Dataset.from_list(dpo_dataset)
new_ds.push_to_hub("Elfsong/Venus_DPO", 'python3', split='runtime')

In [ ]:
dpo_dataset = list()

for instance in tqdm(list(ds)):
    if len(instance["mm_list"]) < 5:
        continue

    for _ in range(512):
        # Randomly sample three solutions
        (solution_1, solution_2, solution_3) = random.sample(instance["mm_list"], 3)

        # If the memory difference is greater than 128 kb, generate a new training instance
        if (int(solution_1['memory']) - int(solution_2['memory']) >= 128) and (int(solution_2['memory']) - int(solution_3['memory']) >= 128):
            sft_instance = {
                "input": prompt_template.format(instruction="Generate a solution with less memory.", content=instance["content"], solution_code=solution_2["code"]),
                "chosen": solution_3["code"],
                "rejected": solution_1["code"]
            }
            dpo_dataset.append(sft_instance)

new_ds = Dataset.from_list(dpo_dataset)
new_ds.push_to_hub("Elfsong/Venus_DPO", 'python3', split='memory')